<a href="https://colab.research.google.com/github/Sjleerodls/Data_Analysis/blob/main/lab_da/da12_shape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DataFrame 모양(shape) 변경

* wide (columns) --> long (rows)
* long (rows) --> wide (columns)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# stack vs unstack

In [148]:
df = pd.DataFrame(data=np.arange(1, 7).reshape((2, 3)),     # reshape으로 행과 열 설정
                  columns=['a', 'b', 'c'],
                  index=['X', 'Y'])
df

,a,b,c
X,1,2,3
Y,4,5,6


In [149]:
df_stacked = df.stack()
df_stacked  #> 컬럼 이름들이 index(row label)로 바뀜. wide --> long

X  a    1
   b    2
   c    3
Y  a    4
   b    5
   c    6
dtype: int64

In [10]:
df_unstacked = df_stacked.unstack()     # index level : -1 기본 설정되어 있음.
df_unstacked    #> 가장 마지막 레벨의 인덱스를 컬럼으로 변환. long --> wide

,a,b,c
X,1,2,3
Y,4,5,6


In [12]:
df_stacked.unstack(level=0)     # 기본 설정이 -1 로 설정되어 있지만 0으로 설정하면 기존 X,Y가 컬럼으로 바뀜.

,X,Y
a,1,4
b,2,5
c,3,6


컬럼 이름(인덱스)가 MultiIndex인 경우

In [17]:
df = pd.DataFrame(data=np.arange(1, 13).reshape(2, 6),
                  columns=[['Lunch'] * 3 + ['Dinner'] * 3,
                           ['Fri', 'Sat', 'Sun'] * 2])
df

Lunch         Dinner        
    Fri Sat Sun    Fri Sat Sun
0     1   2   3      4   5   6
1     7   8   9     10  11  12

In [19]:
df.columns      #> MultiIndex       튜플로 이루어진 배열.

MultiIndex([( 'Lunch', 'Fri'),
            ( 'Lunch', 'Sat'),
            ( 'Lunch', 'Sun'),
            ('Dinner', 'Fri'),
            ('Dinner', 'Sat'),
            ('Dinner', 'Sun')],
           )

In [22]:
df.stack()      # 마지막 레벨의 컬럼만 index로 내려옴.
                #> 가장 마지막 레벨의 컬럼 이름들을 index(row label)로 변환

Lunch  Dinner
0 Fri      1       4
  Sat      2       5
  Sun      3       6
1 Fri      7      10
  Sat      8      11
  Sun      9      12

In [25]:
df.stack(level=[0, 1])      # 리스트로도 사용 가능함.

0  Dinner  Fri     4
           Sat     5
           Sun     6
   Lunch   Fri     1
           Sat     2
           Sun     3
1  Dinner  Fri    10
           Sat    11
           Sun    12
   Lunch   Fri     7
           Sat     8
           Sun     9
dtype: int64

# pivot vs melt

In [28]:
df = pd.DataFrame(data={
    'time': ['Lunch'] * 3 + ['Dinner'] * 3,
    'day': ['Fri', 'Sat', 'Sun'] * 2,
    'tip': np.arange(1,7),
    'total_bill': np.arange(10, 70, 10)
})

df

,time,day,tip,total_bill
0,Lunch,Fri,1,10
1,Lunch,Sat,2,20
2,Lunch,Sun,3,30
3,Dinner,Fri,4,40
4,Dinner,Sat,5,50
5,Dinner,Sun,6,60


## pivot

`pd.DataFrame.pivot()` 메서드 파라미터:
* columns : pivoting 데이터프레임에서 컬럼 이름으로 사용하기 위한 컬럼(들)의 이름.
* index : pivoting 데이터프레임에서 인덱스(row label)로 사용하기 위한 컬럼(들)의 이름.
* values : pivoting 데이터프레임에서 각 셀에 채울 값들을 가지고 있는 컬럼(들)의 이름.

In [30]:
df.pivot(columns='day', index='time', values='tip')

day,Fri,Sat,Sun
time,,,
Dinner,4,5,6
Lunch,1,2,3


In [32]:
df.pivot(columns='time', index='day', values='total_bill')

time,Dinner,Lunch
day,,
Fri,40,10
Sat,50,20
Sun,60,30


## melt

In [34]:
df = pd.DataFrame(data={
    'gender': ['Female', 'Male'],
    'lunch': [10, 7],
    'dinner': [20, 30]
})

df

,gender,lunch,dinner
0,Female,10,20
1,Male,7,30


`pd.DataFrame.melt()` 메서드 파라미터 :
* id_vars : melting 될 때 컬럼으로 유지될 컬럼(들) 이름.
    * id_vars에 설정하지 않은 컬럼 이름들은 variable 컬럼으로 melting됨.
    * variable 컬럼의 값으로 들어감.
    * iv_vars에 설정하지 않은 컬럼들의 모든 셀에 있는 값들은 value 컬럼으로 melting됨.
* var_name : variable 컬럼의 이름을 사용할 문자열.
* value_name : value 컬럼의 이름으로 사용할 문자열.

In [36]:
df.melt(id_vars='gender')

,gender,variable,value
0,Female,lunch,10
1,Male,lunch,7
2,Female,dinner,20
3,Male,dinner,30


In [38]:
df.melt(id_vars='gender', var_name='time')

,gender,time,value
0,Female,lunch,10
1,Male,lunch,7
2,Female,dinner,20
3,Male,dinner,30


In [41]:
df.melt(id_vars='gender', var_name='time', value_name='count')

,gender,time,count
0,Female,lunch,10
1,Male,lunch,7
2,Female,dinner,20
3,Male,dinner,30


# pivot_table

groupby 연산과 통계 함수 적용 결과를 pivoting 하는 함수

In [44]:
tips = sns.load_dataset('tips')

In [45]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


## 성별 팁의 평균

In [46]:
# 성별 팁의 평균을 계산
tips.groupby(by=['sex']).tip.mean()

,tip
sex,
Male,3.089618
Female,2.833448


In [48]:
tips.pivot_table(values='tip', index='sex')     # aggfunc = 'mean' 기본값

,tip
sex,
Male,3.089618
Female,2.833448


## 성별, 흡연여부별 tip의 평균

In [53]:
by_sex_smoker = tips.groupby(by=['sex', 'smoker']).tip.mean()
by_sex_smoker

sex     smoker
Male    Yes       3.051167
        No        3.113402
Female  Yes       2.931515
        No        2.773519
Name: tip, dtype: float64

In [54]:
by_sex_smoker.unstack()

smoker,Yes,No
sex,,
Male,3.051167,3.113402
Female,2.931515,2.773519


In [55]:
tips.pivot_table(values='tip', index=['sex', 'smoker'])

tip
sex    smoker          
Male   Yes     3.051167
       No      3.113402
Female Yes     2.931515
       No      2.773519

In [57]:
tips.pivot_table(values='tip', index='sex', columns='smoker')

smoker,Yes,No
sex,,
Male,3.051167,3.113402
Female,2.931515,2.773519


## 성별 팁, 영수증 평균

In [60]:
by_sex = tips.groupby(by=['sex'])[['tip', 'total_bill']].mean()
by_sex

,tip,total_bill
sex,,
Male,3.089618,20.744076
Female,2.833448,18.056897


In [62]:
tips.pivot_table(values=['tip', 'total_bill'], index='sex')

,tip,total_bill
sex,,
Male,3.089618,20.744076
Female,2.833448,18.056897


## 성별, 흡연여부별 팁, 영수증 평균

In [150]:
by_sex_smoker = tips.groupby(by=['sex', 'smoker'])[['tip', 'total_bill']].mean()
by_sex_smoker

tip  total_bill
sex    smoker                      
Male   Yes     3.051167   22.284500
       No      3.113402   19.791237
Female Yes     2.931515   17.977879
       No      2.773519   18.105185

In [152]:
by_sex_smoker.unstack()

tip           total_bill           
smoker       Yes        No        Yes         No
sex                                             
Male    3.051167  3.113402  22.284500  19.791237
Female  2.931515  2.773519  17.977879  18.105185

In [68]:
tips.pivot_table(values=['tip', 'total_bill'], index=['sex', 'smoker'])

tip  total_bill
sex    smoker                      
Male   Yes     3.051167   22.284500
       No      3.113402   19.791237
Female Yes     2.931515   17.977879
       No      2.773519   18.105185

In [72]:
tips.pivot_table(values=['tip', 'total_bill'], index=['sex'], columns='smoker')

tip           total_bill           
smoker       Yes        No        Yes         No
sex                                             
Male    3.051167  3.113402  22.284500  19.791237
Female  2.931515  2.773519  17.977879  18.105185

## Ex 1. 성별, 요일별, 시간별 팁의 평균

In [104]:
# groupby
result = tips.groupby(by=['sex', 'day', 'time']).tip.mean()
result

sex     day   time  
Male    Thur  Lunch     2.980333
              Dinner         NaN
        Fri   Lunch     1.900000
              Dinner    3.032857
        Sat   Lunch          NaN
              Dinner    3.083898
        Sun   Lunch          NaN
              Dinner    3.220345
Female  Thur  Lunch     2.561935
              Dinner    3.000000
        Fri   Lunch     2.745000
              Dinner    2.810000
        Sat   Lunch          NaN
              Dinner    2.801786
        Sun   Lunch          NaN
              Dinner    3.367222
Name: tip, dtype: float64

In [106]:
result.unstack()

time            Lunch    Dinner
sex    day                     
Male   Thur  2.980333       NaN
       Fri   1.900000  3.032857
       Sat        NaN  3.083898
       Sun        NaN  3.220345
Female Thur  2.561935  3.000000
       Fri   2.745000  2.810000
       Sat        NaN  2.801786
       Sun        NaN  3.367222

In [108]:
result.unstack(level=1)

day                Thur       Fri       Sat       Sun
sex    time                                          
Male   Lunch   2.980333  1.900000       NaN       NaN
       Dinner       NaN  3.032857  3.083898  3.220345
Female Lunch   2.561935  2.745000       NaN       NaN
       Dinner  3.000000  2.810000  2.801786  3.367222

In [109]:
# pivot_table
tips.pivot_table(values='tip', index=['sex', 'day', 'time'])

tip
sex    day  time            
Male   Thur Lunch   2.980333
       Fri  Lunch   1.900000
            Dinner  3.032857
       Sat  Dinner  3.083898
       Sun  Dinner  3.220345
Female Thur Lunch   2.561935
            Dinner  3.000000
       Fri  Lunch   2.745000
            Dinner  2.810000
       Sat  Dinner  2.801786
       Sun  Dinner  3.367222

In [111]:
tips.pivot_table(values='tip', index=['sex', 'day'], columns='time')

time            Lunch    Dinner
sex    day                     
Male   Thur  2.980333       NaN
       Fri   1.900000  3.032857
       Sat        NaN  3.083898
       Sun        NaN  3.220345
Female Thur  2.561935  3.000000
       Fri   2.745000  2.810000
       Sat        NaN  2.801786
       Sun        NaN  3.367222

In [112]:
tips.pivot_table(values='tip', index=['sex', 'time'], columns='day')

day                Thur       Fri       Sat       Sun
sex    time                                          
Male   Lunch   2.980333  1.900000       NaN       NaN
       Dinner       NaN  3.032857  3.083898  3.220345
Female Lunch   2.561935  2.745000       NaN       NaN
       Dinner  3.000000  2.810000  2.801786  3.367222

## Ex 2. 성별 팁의 최솟값, 중위값, 최댓값

In [116]:
# groupby
min_by_tip = tips.groupby(by=['sex']).tip.min()
max_by_tip = tips.groupby(by=['sex']).tip.max()
median_by_tip = tips.groupby(by=['sex']).tip.median()

print(min_by_tip)
print(max_by_tip)
print(median_by_tip)

sex
Male      1.0
Female    1.0
Name: tip, dtype: float64
sex
Male      10.0
Female     6.5
Name: tip, dtype: float64
sex
Male      3.00
Female    2.75
Name: tip, dtype: float64


In [119]:
tips.groupby(by=['sex']).tip.agg(['min', 'median', 'max'])

,min,median,max
sex,,,
Male,1.0,3.00,10.0
Female,1.0,2.75,6.5


In [115]:
# pivot_table
tips.pivot_table(values='tip', index='sex', aggfunc=['min', 'median', 'max'])

,min,median,max
,tip,tip,tip
sex,,,
Male,1.0,3.00,10.0
Female,1.0,2.75,6.5


## Ex 3. 성별, 요일별 영수증 최솟값, 중위값, 최댓값

In [120]:
# groupby
tips.groupby(by=['sex', 'day']).total_bill.min()
tips.groupby(by=['sex', 'day']).total_bill.median()
tips.groupby(by=['sex', 'day']).total_bill.max()

sex     day 
Male    Thur    41.19
        Fri     40.17
        Sat     50.81
        Sun     48.17
Female  Thur    43.11
        Fri     22.75
        Sat     44.30
        Sun     35.26
Name: total_bill, dtype: float64

In [122]:
result = tips.groupby(by=['sex', 'day']).tip.agg(['min', 'median', 'max'])
result

min  median    max
sex    day                      
Male   Thur  1.44   2.530   6.70
       Fri   1.50   2.600   4.73
       Sat   1.00   3.000  10.00
       Sun   1.32   3.085   6.50
Female Thur  1.25   2.005   5.17
       Fri   1.00   3.000   4.30
       Sat   1.00   2.625   6.50
       Sun   1.01   3.500   5.20

In [123]:
result.unstack()

min                 median                      max                 
day     Thur  Fri  Sat   Sun   Thur  Fri    Sat    Sun  Thur   Fri   Sat  Sun
sex                                                                          
Male    1.44  1.5  1.0  1.32  2.530  2.6  3.000  3.085  6.70  4.73  10.0  6.5
Female  1.25  1.0  1.0  1.01  2.005  3.0  2.625  3.500  5.17  4.30   6.5  5.2

In [125]:
result.unstack(level=0)

min        median           max       
sex   Male Female   Male Female   Male Female
day                                          
Thur  1.44   1.25  2.530  2.005   6.70   5.17
Fri   1.50   1.00  2.600  3.000   4.73   4.30
Sat   1.00   1.00  3.000  2.625  10.00   6.50
Sun   1.32   1.01  3.085  3.500   6.50   5.20

In [127]:
# pivot_table
tips.pivot_table(values='total_bill', index=['sex', 'day'], aggfunc=['min', 'median', 'max'])

min     median        max
            total_bill total_bill total_bill
sex    day                                  
Male   Thur       7.51     16.975      41.19
       Fri        8.58     17.215      40.17
       Sat        7.74     18.240      50.81
       Sun        7.25     20.725      48.17
Female Thur       8.35     13.785      43.11
       Fri        5.75     15.380      22.75
       Sat        3.07     18.360      44.30
       Sun        9.60     17.410      35.26

In [129]:
tips.pivot_table(values='total_bill', index='sex', columns= 'day', aggfunc=['min', 'median', 'max'])

min                    median                           max         \
day     Thur   Fri   Sat   Sun    Thur     Fri    Sat     Sun   Thur    Fri   
sex                                                                           
Male    7.51  8.58  7.74  7.25  16.975  17.215  18.24  20.725  41.19  40.17   
Female  8.35  5.75  3.07  9.60  13.785  15.380  18.36  17.410  43.11  22.75   

                      
day       Sat    Sun  
sex                   
Male    50.81  48.17  
Female  44.30  35.26

In [130]:
tips.pivot_table(values='total_bill', index='day', columns= 'sex', aggfunc=['min', 'median', 'max'])

min         median            max       
sex   Male Female    Male  Female   Male Female
day                                            
Thur  7.51   8.35  16.975  13.785  41.19  43.11
Fri   8.58   5.75  17.215  15.380  40.17  22.75
Sat   7.74   3.07  18.240  18.360  50.81  44.30
Sun   7.25   9.60  20.725  17.410  48.17  35.26

## Ex 4. 성별, 흡연여부별, 요일별 팁의 중위값

In [132]:
# groupby
result = tips.groupby(by=['sex', 'smoker', 'day']).tip.median()
result

sex     smoker  day 
Male    Yes     Thur    2.780
                Fri     2.600
                Sat     3.000
                Sun     3.500
        No      Thur    2.405
                Fri     2.500
                Sat     2.860
                Sun     3.000
Female  Yes     Thur    2.500
                Fri     2.500
                Sat     2.500
                Sun     3.500
        No      Thur    2.000
                Fri     3.125
                Sat     2.750
                Sun     3.500
Name: tip, dtype: float64

In [134]:
result.unstack()

day             Thur    Fri   Sat  Sun
sex    smoker                         
Male   Yes     2.780  2.600  3.00  3.5
       No      2.405  2.500  2.86  3.0
Female Yes     2.500  2.500  2.50  3.5
       No      2.000  3.125  2.75  3.5

In [135]:
result.unstack(level=1)

smoker        Yes     No
sex    day              
Male   Thur  2.78  2.405
       Fri   2.60  2.500
       Sat   3.00  2.860
       Sun   3.50  3.000
Female Thur  2.50  2.000
       Fri   2.50  3.125
       Sat   2.50  2.750
       Sun   3.50  3.500

In [136]:
result.unstack(level=[0, 1])

sex     Male        Female       
smoker   Yes     No    Yes     No
day                              
Thur    2.78  2.405    2.5  2.000
Fri     2.60  2.500    2.5  3.125
Sat     3.00  2.860    2.5  2.750
Sun     3.50  3.000    3.5  3.500

In [98]:
# pivot_table
tips.pivot_table(values='tip', index=['sex', 'smoker', 'day'], aggfunc='median')

tip
sex    smoker day        
Male   Yes    Thur  2.780
              Fri   2.600
              Sat   3.000
              Sun   3.500
       No     Thur  2.405
              Fri   2.500
              Sat   2.860
              Sun   3.000
Female Yes    Thur  2.500
              Fri   2.500
              Sat   2.500
              Sun   3.500
       No     Thur  2.000
              Fri   3.125
              Sat   2.750
              Sun   3.500

## Ex 5. 성별, 흡연여부별, 요일별, 시간별 팁의 중앙값

In [138]:
# groupby
result = tips.groupby(by=['sex', 'smoker', 'day', 'time']).tip.median()
result

sex     smoker  day   time  
Male    Yes     Thur  Lunch     2.780
                      Dinner      NaN
                Fri   Lunch     1.920
                      Dinner    3.000
                Sat   Lunch       NaN
                      Dinner    3.000
                Sun   Lunch       NaN
                      Dinner    3.500
        No      Thur  Lunch     2.405
                      Dinner      NaN
                Fri   Lunch       NaN
                      Dinner    2.500
                Sat   Lunch       NaN
                      Dinner    2.860
                Sun   Lunch       NaN
                      Dinner    3.000
Female  Yes     Thur  Lunch     2.500
                      Dinner      NaN
                Fri   Lunch     2.500
                      Dinner    2.750
                Sat   Lunch       NaN
                      Dinner    2.500
                Sun   Lunch       NaN
                      Dinner    3.500
        No      Thur  Lunch     2.000
                      Dinner    3.000
                Fri   Lunch     3.000
                      Dinner    3.250
                Sat   Lunch       NaN
                      Dinner    2.750
                Sun   Lunch       NaN
                      Dinner    3.500
Name: tip, dtype: float64

In [139]:
result.unstack()

time                Lunch  Dinner
sex    smoker day                
Male   Yes    Thur  2.780     NaN
              Fri   1.920    3.00
              Sat     NaN    3.00
              Sun     NaN    3.50
       No     Thur  2.405     NaN
              Fri     NaN    2.50
              Sat     NaN    2.86
              Sun     NaN    3.00
Female Yes    Thur  2.500     NaN
              Fri   2.500    2.75
              Sat     NaN    2.50
              Sun     NaN    3.50
       No     Thur  2.000    3.00
              Fri   3.000    3.25
              Sat     NaN    2.75
              Sun     NaN    3.50

In [142]:
result.unstack(level=2)

day                    Thur   Fri   Sat  Sun
sex    smoker time                          
Male   Yes    Lunch   2.780  1.92   NaN  NaN
              Dinner    NaN  3.00  3.00  3.5
       No     Lunch   2.405   NaN   NaN  NaN
              Dinner    NaN  2.50  2.86  3.0
Female Yes    Lunch   2.500  2.50   NaN  NaN
              Dinner    NaN  2.75  2.50  3.5
       No     Lunch   2.000  3.00   NaN  NaN
              Dinner  3.000  3.25  2.75  3.5

In [143]:
result.unstack(level=['day', 'time'])

day             Thur          Fri          Sat          Sun       
time           Lunch Dinner Lunch Dinner Lunch Dinner Lunch Dinner
sex    smoker                                                     
Male   Yes     2.780    NaN  1.92   3.00   NaN   3.00   NaN    3.5
       No      2.405    NaN   NaN   2.50   NaN   2.86   NaN    3.0
Female Yes     2.500    NaN  2.50   2.75   NaN   2.50   NaN    3.5
       No      2.000    3.0  3.00   3.25   NaN   2.75   NaN    3.5

In [144]:
# pivot_table
tips.pivot_table(values='tip', index=['sex', 'smoker', 'day', 'time'], aggfunc='median')

tip
sex    smoker day  time         
Male   Yes    Thur Lunch   2.780
              Fri  Lunch   1.920
                   Dinner  3.000
              Sat  Dinner  3.000
              Sun  Dinner  3.500
       No     Thur Lunch   2.405
              Fri  Dinner  2.500
              Sat  Dinner  2.860
              Sun  Dinner  3.000
Female Yes    Thur Lunch   2.500
              Fri  Lunch   2.500
                   Dinner  2.750
              Sat  Dinner  2.500
              Sun  Dinner  3.500
       No     Thur Lunch   2.000
                   Dinner  3.000
              Fri  Lunch   3.000
                   Dinner  3.250
              Sat  Dinner  2.750
              Sun  Dinner  3.500

In [146]:
tips.pivot_table(values='tip', index=['sex', 'smoker'], columns=['day', 'time'], aggfunc='median')

day             Thur          Fri           Sat    Sun
time           Lunch Dinner Lunch Dinner Dinner Dinner
sex    smoker                                         
Male   Yes     2.780    NaN  1.92   3.00   3.00    3.5
       No      2.405    NaN   NaN   2.50   2.86    3.0
Female Yes     2.500    NaN  2.50   2.75   2.50    3.5
       No      2.000    3.0  3.00   3.25   2.75    3.5

In [147]:
tips.pivot_table(values='tip', index=['day', 'time'], columns=['sex', 'smoker'], aggfunc='median')

sex          Male        Female      
smoker        Yes     No    Yes    No
day  time                            
Thur Lunch   2.78  2.405   2.50  2.00
     Dinner   NaN    NaN    NaN  3.00
Fri  Lunch   1.92    NaN   2.50  3.00
     Dinner  3.00  2.500   2.75  3.25
Sat  Dinner  3.00  2.860   2.50  2.75
Sun  Dinner  3.50  3.000   3.50  3.50